This experiment, higher learning rate. Larger rollouts.

In [1]:
import deep_rl

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.nn import functional as F
from torch.autograd import Variable
from torch import nn, optim
import torch.utils.data

# load as dask array
import time

import logging
import sys
import os
import glob
import numpy as np
import datetime
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [4]:
from deep_rl.utils.logger import get_logger, get_default_log_dir

from deep_rl.network.network_heads import CategoricalActorCriticNet, QuantileNet, OptionCriticNet, DeterministicActorCriticNet, GaussianActorCriticNet
from deep_rl.network.network_bodies import FCBody
from deep_rl.utils.normalizer import RunningStatsNormalizer, RescaleNormalizer
from deep_rl.component.task import ParallelizedTask

In [5]:
from world_models_sonic.models.vae import VAE5, loss_function_vae
from world_models_sonic.helpers.summarize import TorchSummarizeDf
from world_models_sonic.models.rnn import MDNRNN
from world_models_sonic.models.inverse_model import InverseModel2
from world_models_sonic.models.world_model import WorldModel
from world_models_sonic.custom_envs.env import make_env
from world_models_sonic.custom_envs.wrappers import RandomGameReset
from world_models_sonic import config
from world_models_sonic.helpers.deep_rl import PPOAgent, run_iterations, SonicWorldModelDeepRL, CategoricalWorldActorCriticNet, Config

Importing 0 potential games...
Imported 0 games


# Init

In [6]:

env_name = 'sonic256'
z_dim = 512  # latent dimensions
channels = 3

# RNN
action_dim = 10
image_size = 128

verbose = True  # Set this true to render (and make it go slower)

NAME = 'RNN_v3b_128im_512z_1512_v6k_VAE5_all_noframestack'
ppo_save_file = './outputs/{NAME}/PPO_512z_all_g.pkl'.format(NAME=NAME)
cuda = torch.cuda.is_available()

In [7]:
# Logging
if not os.path.isdir('./outputs/{NAME}'.format(NAME=NAME)):
    os.makedirs('./outputs/{NAME}'.format(NAME=NAME))

# Log to file and stream
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(NAME)

log_dir = log_dir='./outputs/{NAME}'.format(NAME=NAME)
print(log_dir)

deep_rl_logger = get_logger(
    NAME[-10:],
    file_name='deep_rl_ppo.log',
    level=logging.INFO,
    log_dir='./outputs/{NAME}'.format(NAME=NAME), )

./outputs/RNN_v3b_128im_512z_1512_v6k_VAE5_all_noframestack


# World model

In [8]:
# Load VAE
# TODO swap z and k dim, since it's inconsistent with other models
vae = VAE5(image_size=image_size, z_dim=128, conv_dim=64, code_dim=8, k_dim=z_dim, channels=channels)
    
# Load MDRNN
action_dim, hidden_size, n_mixture, temp = action_dim, z_dim*2, 5, 1.0

mdnrnn = MDNRNN(z_dim, action_dim, hidden_size, n_mixture, temp)
    
finv = InverseModel2(z_dim, action_dim, hidden_size=64)
    
world_model = WorldModel(vae, mdnrnn, finv, logger=deep_rl_logger, lambda_vae_kld=1 / 12., lambda_finv=1/200, lambda_vae=1/8, lambda_loss=1000)
world_model = world_model.train()
if cuda:
    world_model = world_model.cuda()

In [9]:
# Optimiser for world models
import torch.optim.lr_scheduler
torch.cuda.empty_cache()
optimizer = optim.Adam(world_model.parameters(), lr=4e-5)

world_model.optimizer = optimizer

# Train

In [10]:
# batch: We have two differen't batch sizes, determined by your gpu:
# VAE: rollout_length * world_model_batch_size which is how much VAE data you can get in your gpu, for my 8GB gpu it was ~50
# PPO: (config.rollout_length*config.num_workers)/config.num_mini_batches: how much PPO data you can git for me it was ~500


z_state_dim=world_model.mdnrnn.z_dim + world_model.mdnrnn.hidden_size


def task_fn(log_dir):
    return SonicWorldModelDeepRL(
        env_fn=lambda: RandomGameReset(make_env(
            'sonic', max_episode_steps=4000, to_gray=False, image_size=image_size)),
        log_dir=log_dir,
        verbose=verbose
    )

config = Config()

verbose = False  # Set this true to render (and make it go slower)
config.num_workers = 1 if verbose else 10
config.task_fn = lambda: ParallelizedTask(
    task_fn, config.num_workers, single_process=config.num_workers == 1)
config.optimizer_fn = lambda params: torch.optim.Adam(params, 1e-4, eps=1e-6)
config.network_fn = lambda state_dim, action_dim: CategoricalWorldActorCriticNet(
    state_dim, action_dim, FCBody(z_state_dim, hidden_units=(64, 64), gate=F.relu), gpu=0 if cuda else -1, world_model_fn=lambda: world_model,
    render=(config.num_workers==1 and verbose),
    z_shape=(32, 16)
)
# see params here https://github.com/openai/retro-baselines/blob/master/agents/ppo2_agent.py
config.discount = 0.99
config.logger = deep_rl_logger
config.use_gae = True
config.gae_tau = 0.95
config.entropy_weight = 0.02
config.value_weight = 0.5
config.gradient_clip = 0.5  # Watch grad_norm in tensorboard and set this to upper quartile to avoid clipping too much
config.rollout_length = 36
config.optimization_epochs = 3
config.num_mini_batches = 8
config.ppo_ratio_clip = 0.1
config.iteration_log_interval = 1

config.train_world_model = False
if not config.train_world_model:
    config.rollout_length*=12  # More can fit in batch
config.world_model_batch_size = 2

# I tuned these so the intrinsic reward was 1) within an order of magnitude of the extrinsic. 2) smaller, 3) negative when stuck
# TODO use reward normalisers etc to reduce the need for these hyperparameters
config.curiosity = False
config.curiosity_only = False
config.curiosity_weight = 0.01
config.curiosity_boredom = 2 # how many standard deviations above the mean does it's new experience need to be, so it's not bored
# config.reward_normalizer = RunningStatsNormalizer()
# config.intrinsic_reward_normalizer = RunningStatsNormalizer()
agent = PPOAgent(config)

print(('total rollout of ', config.rollout_length*config.num_workers))
print(('made of sequences of length', (config.rollout_length)))
print(('ppo mini batch', (config.rollout_length*config.num_workers)//config.num_mini_batches))


if os.path.isfile(ppo_save_file):
    print(('loading ppo_save_file', ppo_save_file, 'modified', time.ctime(os.path.getmtime(ppo_save_file))))
    agent.load(ppo_save_file)
    
#     # also load normalizers
    state_dict = torch.load(ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
    config.intrinsic_reward_normalizer.load_state_dict(state_dict)

    state_dict = torch.load(ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))
    config.reward_normalizer.load_state_dict(state_dict)
else:
    print("couldn't find save")

game: SonicAndKnuckles3-Genesis state: MarbleGardenZone.Act1
game: SonicTheHedgehog2-Genesis state: EmeraldHillZone.Act2
game: SonicTheHedgehog-Genesis state: ScrapBrainZone.Act2
game: SonicTheHedgehog2-Genesis state: MetropolisZone.Act1
game: SonicTheHedgehog-Genesis state: SpringYardZone.Act2
game: SonicAndKnuckles3-Genesis state: SandopolisZone.Act1
game: SonicAndKnuckles3-Genesis state: DeathEggZone.Act1
game: SonicTheHedgehog2-Genesis state: EmeraldHillZone.Act1
game: SonicAndKnuckles3-Genesis state: AngelIslandZone.Act1
game: SonicAndKnuckles3-Genesis state: DeathEggZone.Act1
('total rollout of ', 4320)
('made of sequences of length', 432)
('ppo mini batch', 540)
('loading ppo_save_file', './outputs/RNN_v3b_128im_512z_1512_v6k_VAE5_all_noframestack/PPO_512z_all_g.pkl', 'modified', 'Fri Jun 22 16:57:50 2018')


Process ProcessWrapper-8:
Process ProcessWrapper-4:
Process ProcessWrapper-5:
Traceback (most recent call last):
Process ProcessWrapper-7:
Traceback (most recent call last):
Traceback (most recent call last):
Process ProcessWrapper-9:
Process ProcessWrapper-2:
  File "/home/wassname/.pyenv/versions/3.5.3/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
Process ProcessWrapper-6:
Process ProcessWrapper-3:
Process ProcessWrapper-1:
  File "/home/wassname/.pyenv/versions/3.5.3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/wassname/.pyenv/versions/3.5.3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/wassname/.pyenv/versions/3.5.3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/media/oldhome/wassname/Documents/projects/retro_sonic_comp/DeepRL/deep_rl/component/task.py", line 177, in run
    op, data = self.pi

In [11]:
# # DEBUG

# # reset from checkpoint
# agent.load('./outputs/RNN_v3b_128im_512z_1512_v6j_VAE5_all/PPO_512z_all_g-20180606_02-06-59.pkl')

# # # Reset just rnn
# world_model.mdnrnn = MDNRNN(z_dim, action_dim, hidden_size, n_mixture, temp)
# world_model.mdnrnn.cuda()

# # if we want to reset the actor
# from deep_rl.network.network_heads import ActorCriticNet
# agent.network.network = ActorCriticNet(agent.network.z_state_dim, action_dim, FCBody(z_state_dim, hidden_units=(64, 64), gate=F.relu), None, None)
# agent.network.network.cuda()


# world_model.finv = finv = InverseModel(z_dim, action_dim, hidden_size=z_dim*2)
# world_model.finv.cuda()

In [12]:
# agent.load('/home/wassname/Documents/projects/retro_sonic_comp/world-models-pytorch/outputs/RNN_v3b_128im_512z_1512_v6k_VAE5_all_noframestack/PPO_512z_all_g-20180622_06-38-24.pkl')

In [13]:
print(""""
# To monitor with tensorboard at http://localhost:6006/
cd ./outputs/{NAME}/
tensorboard  --logdir .
""".format(NAME=NAME))

"
# To monitor with tensorboard at http://localhost:6006/
cd ./outputs/RNN_v3b_128im_512z_1512_v6k_VAE5_all_noframestack/
tensorboard  --logdir .



In [14]:
try:
    run_iterations(agent, log_dir=log_dir)
except KeyboardInterrupt:
    # Close the environment
    if config.num_workers == 1:
        agent.task.tasks[0].env.close()
    else:
        [t.close() for t in agent.task.tasks]

    print("saving", ppo_save_file)
    agent.save(ppo_save_file)
#     torch.save(config.intrinsic_reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
#     torch.save(config.reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))

    # Backup since it sometimes get's corrupted
    ts = datetime.datetime.utcnow().strftime('%Y%m%d_%H-%M-%S')
    print("saving backup",
          ppo_save_file.replace('.pkl', '-%s.pkl' % ts),)
    agent.save(ppo_save_file.replace('.pkl', '-%s.pkl' % ts))
    raise

2018-06-22 16:58:54,533 - framestack - INFO: action counts: {0: 334, 1: 89, 2: 1027, 3: 46, 4: 1041, 5: 200, 6: 282, 7: 73, 8: 893, 9: 335}
2018-06-22 16:58:54,558 - framestack - INFO: steps: 4320, steps/s: 88.58
  epoch reward:       0.0000/1.2000/7.9751 [n=10] (min/mean/max)
  running reward:     0.0000/1.2000/7.9751 [n=10]

2018-06-22 16:59:44,564 - framestack - INFO: action counts: {0: 317, 1: 91, 2: 1105, 3: 45, 4: 1009, 5: 230, 6: 231, 7: 79, 8: 902, 9: 311}
2018-06-22 16:59:44,590 - framestack - INFO: steps: 8640, steps/s: 87.46
  epoch reward:       0.0000/1.2000/7.9751 [n=10] (min/mean/max)
  running reward:     0.0000/1.2000/7.9751 [n=20]

2018-06-22 17:00:34,102 - framestack - INFO: action counts: {0: 316, 1: 83, 2: 1063, 3: 35, 4: 1028, 5: 218, 6: 262, 7: 65, 8: 891, 9: 359}
2018-06-22 17:00:34,124 - framestack - INFO: steps: 12960, steps/s: 87.38
  epoch reward:       0.0000/1.2000/7.9751 [n=10] (min/mean/max)
  running reward:     0.0000/1.2000/7.9751 [n=30]

2018-06-22 1

2018-06-22 17:19:41,007 - framestack - INFO: action counts: {0: 319, 1: 112, 2: 974, 3: 61, 4: 975, 5: 253, 6: 313, 7: 112, 8: 828, 9: 373}
2018-06-22 17:19:41,032 - framestack - INFO: steps: 112320, steps/s: 86.74
  epoch reward:       2.0606/8.7370/26.6185 [n=10] (min/mean/max)
  running reward:     0.0000/6.9152/26.6303 [n=260]

2018-06-22 17:20:32,032 - framestack - INFO: action counts: {0: 321, 1: 99, 2: 1013, 3: 74, 4: 948, 5: 244, 6: 319, 7: 113, 8: 825, 9: 364}
2018-06-22 17:20:32,063 - framestack - INFO: steps: 116640, steps/s: 86.67
  epoch reward:       2.0606/8.7370/26.6185 [n=10] (min/mean/max)
  running reward:     0.0000/6.9826/26.6303 [n=270]

2018-06-22 17:21:20,807 - framestack - INFO: action counts: {0: 305, 1: 118, 2: 986, 3: 72, 4: 977, 5: 214, 6: 294, 7: 98, 8: 877, 9: 379}
2018-06-22 17:21:20,829 - framestack - INFO: steps: 120960, steps/s: 86.74
  epoch reward:       2.8419/7.4618/26.6185 [n=10] (min/mean/max)
  running reward:     0.0000/6.9997/26.6303 [n=280]


2018-06-22 17:40:36,117 - framestack - INFO: action counts: {0: 290, 1: 76, 2: 1133, 3: 41, 4: 1091, 5: 181, 6: 218, 7: 68, 8: 920, 9: 302}
2018-06-22 17:40:36,140 - framestack - INFO: steps: 220320, steps/s: 86.43
  epoch reward:       1.8440/12.4674/27.9323 [n=10] (min/mean/max)
  running reward:     0.0000/8.7931/28.3140 [n=500]

2018-06-22 17:41:24,722 - framestack - INFO: action counts: {0: 250, 1: 54, 2: 1177, 3: 32, 4: 1072, 5: 194, 6: 222, 7: 59, 8: 955, 9: 305}
2018-06-22 17:41:24,746 - framestack - INFO: steps: 224640, steps/s: 86.47
  epoch reward:       1.8440/11.5406/27.9323 [n=10] (min/mean/max)
  running reward:     0.0000/8.9999/28.3140 [n=500]

2018-06-22 17:42:12,597 - framestack - INFO: action counts: {0: 252, 1: 76, 2: 1183, 3: 35, 4: 1047, 5: 175, 6: 242, 7: 58, 8: 929, 9: 323}
2018-06-22 17:42:12,620 - framestack - INFO: steps: 228960, steps/s: 86.55
  epoch reward:       1.8440/11.5406/27.9323 [n=10] (min/mean/max)
  running reward:     0.0000/9.2067/28.3140 [n=5

2018-06-22 18:00:48,975 - framestack - INFO: action counts: {0: 211, 1: 55, 2: 1305, 3: 18, 4: 1083, 5: 161, 6: 199, 7: 29, 8: 961, 9: 298}
2018-06-22 18:00:49,003 - framestack - INFO: steps: 328320, steps/s: 87.30
  epoch reward:       2.9088/9.3046/26.1445 [n=10] (min/mean/max)
  running reward:     0.7183/10.8482/28.3140 [n=500]

2018-06-22 18:01:37,013 - framestack - INFO: action counts: {0: 179, 1: 56, 2: 1358, 3: 21, 4: 1071, 5: 117, 6: 186, 7: 39, 8: 1011, 9: 282}
2018-06-22 18:01:37,040 - framestack - INFO: steps: 332640, steps/s: 87.34
  epoch reward:       2.9088/8.8037/26.1445 [n=10] (min/mean/max)
  running reward:     0.7183/10.8495/28.3140 [n=500]

2018-06-22 18:02:25,382 - framestack - INFO: action counts: {0: 194, 1: 44, 2: 1345, 3: 14, 4: 1152, 5: 141, 6: 186, 7: 36, 8: 951, 9: 257}
2018-06-22 18:02:25,404 - framestack - INFO: steps: 336960, steps/s: 87.36
  epoch reward:       2.9088/8.8037/26.1445 [n=10] (min/mean/max)
  running reward:     0.7183/10.8763/28.3140 [n=

2018-06-22 18:20:21,739 - framestack - INFO: steps: 432000, steps/s: 87.57
  epoch reward:       3.2713/12.1956/45.7083 [n=10] (min/mean/max)
  running reward:     0.7183/10.7414/45.7083 [n=500]

2018-06-22 18:21:09,543 - framestack - INFO: action counts: {0: 258, 1: 54, 2: 1214, 3: 25, 4: 1172, 5: 140, 6: 190, 7: 46, 8: 915, 9: 306}
2018-06-22 18:21:09,570 - framestack - INFO: steps: 436320, steps/s: 87.60
  epoch reward:       3.2713/13.2667/45.7083 [n=10] (min/mean/max)
  running reward:     0.7183/10.7574/45.7083 [n=500]

2018-06-22 18:21:56,162 - framestack - INFO: action counts: {0: 229, 1: 47, 2: 1208, 3: 30, 4: 1171, 5: 165, 6: 203, 7: 45, 8: 932, 9: 290}
2018-06-22 18:21:56,190 - framestack - INFO: steps: 440640, steps/s: 87.65
  epoch reward:       5.1114/14.2116/45.7083 [n=10] (min/mean/max)
  running reward:     0.7183/10.8108/45.7083 [n=500]

2018-06-22 18:22:41,961 - framestack - INFO: action counts: {0: 230, 1: 38, 2: 1224, 3: 26, 4: 1159, 5: 154, 6: 210, 7: 44, 8: 940, 

...

2018-06-23 15:42:17,734 - framestack - INFO: action counts: {0: 242, 1: 41, 2: 989, 3: 44, 4: 995, 5: 97, 6: 113, 7: 60, 8: 1359, 9: 380}
2018-06-23 15:42:17,761 - framestack - INFO: steps: 7659360, steps/s: 93.96
  epoch reward:       1.3934/7.7370/27.9265 [n=10] (min/mean/max)
  running reward:     0.7183/11.9027/47.2714 [n=500]

2018-06-23 15:43:02,121 - framestack - INFO: action counts: {0: 248, 1: 29, 2: 992, 3: 39, 4: 971, 5: 102, 6: 113, 7: 53, 8: 1372, 9: 401}
2018-06-23 15:43:02,146 - framestack - INFO: steps: 7663680, steps/s: 93.96
  epoch reward:       1.3934/7.7370/27.9265 [n=10] (min/mean/max)
  running reward:     0.7183/11.8356/47.2714 [n=500]

2018-06-23 15:43:48,854 - framestack - INFO: action counts: {0: 276, 1: 49, 2: 947, 3: 38, 4: 998, 5: 85, 6: 135, 7: 68, 8: 1321, 9: 403}
2018-06-23 15:43:48,880 - framestack - INFO: steps: 7668000, steps/s: 93.96
  epoch reward:       1.6186/9.0730/27.9265 [n=10] (min/mean/max)
  running reward:     0.7183/11.8091/47.2714 [n=500

2018-06-23 16:01:24,161 - framestack - INFO: action counts: {0: 254, 1: 58, 2: 904, 3: 42, 4: 946, 5: 89, 6: 123, 7: 71, 8: 1438, 9: 395}
2018-06-23 16:01:24,192 - framestack - INFO: steps: 7767360, steps/s: 93.97
  epoch reward:       3.0624/8.8993/15.2427 [n=10] (min/mean/max)
  running reward:     0.4673/9.7741/30.8297 [n=500]

2018-06-23 16:02:09,245 - framestack - INFO: action counts: {0: 216, 1: 36, 2: 981, 3: 28, 4: 1019, 5: 89, 6: 129, 7: 55, 8: 1406, 9: 361}
2018-06-23 16:02:09,273 - framestack - INFO: steps: 7771680, steps/s: 93.98
  epoch reward:       3.0624/8.3974/15.2427 [n=10] (min/mean/max)
  running reward:     0.4673/9.7120/30.8297 [n=500]

2018-06-23 16:02:56,348 - framestack - INFO: action counts: {0: 226, 1: 50, 2: 920, 3: 33, 4: 1048, 5: 82, 6: 118, 7: 55, 8: 1417, 9: 371}
2018-06-23 16:02:56,377 - framestack - INFO: steps: 7776000, steps/s: 93.97
  epoch reward:       3.0624/11.1379/27.9336 [n=10] (min/mean/max)
  running reward:     0.4673/9.6995/30.8297 [n=500]

2018-06-23 16:19:40,211 - framestack - INFO: steps: 7871040, steps/s: 94.03
  epoch reward:       2.5548/10.3255/28.4084 [n=10] (min/mean/max)
  running reward:     0.1048/10.3434/47.0356 [n=500]



saving ./outputs/RNN_v3b_128im_512z_1512_v6k_VAE5_all_noframestack/PPO_512z_all_g.pkl
saving backup ./outputs/RNN_v3b_128im_512z_1512_v6k_VAE5_all_noframestack/PPO_512z_all_g-20180623_08-20-24.pkl


KeyboardInterrupt: 

    2018-06-20 09:40:21,906 - framestack - INFO: action counts: {0: 127, 1: 41, 2: 196, 3: 22, 4: 185, 5: 96, 6: 97, 7: 35, 8: 238, 9: 83}
    2018-06-20 09:40:21,928 - framestack - INFO: steps: 23520, steps/s: 47.68
      world model losses: 40.4108 rnn=26.8570, inv= 0.1192=0.0050 * 23.8446, vae=13.4345=0.1250 * (93.5157 + 0.0833 * 167.5278)
      epoch reward:       0.0000/1.2111/12.7297 [n=40] (min/mean/max)
      running reward:     0.0000/0.9334/12.7297 [n=500]

    2018-06-21 07:12:46,362 - framestack - INFO: action counts: {0: 82, 1: 20, 2: 252, 3: 19, 4: 217, 5: 61, 6: 129, 7: 48, 8: 224, 9: 68}
    2018-06-21 07:12:46,379 - framestack - INFO: steps: 3764320, steps/s: 48.60
      world model losses: 34.7660 rnn=24.8725, inv= 0.0004=0.0050 * 0.0753, vae=9.8931=0.1250 * (63.7831 + 0.0833 * 184.3440)
      epoch reward:       0.7183/10.4729/38.4701 [n=40] (min/mean/max)
      running reward:     0.7183/10.6002/38.4701 [n=500]


In [ ]:
agent.save(ppo_save_file)
torch.save(config.intrinsic_reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
torch.save(config.reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))

In [ ]:
agent.save(ppo_save_file)
# torch.save(config.intrinsic_reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
# torch.save(config.reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))

# Summarize model

In [ ]:
agent.network.world_model.mdnrnn.training

In [ ]:
from IPython.display import display

with torch.no_grad():
    img = np.random.randn(image_size, image_size, 3)
    action = np.array(np.random.randint(0,action_dim))[np.newaxis]
    action = Variable(torch.from_numpy(action)).float().cuda()[np.newaxis]
    gpu_img = Variable(torch.from_numpy(img[np.newaxis].transpose(0, 3, 1, 2))).float().cuda()
    if cuda:
        gpu_img = gpu_img.cuda()
    with TorchSummarizeDf(vae) as tdf:
        x, mu_vae, logvar_vae = vae.forward(gpu_img)
        z = vae.sample(mu_vae, logvar_vae)
        df_vae = tdf.make_df()

    display(df_vae[df_vae.level<2])
    
    with TorchSummarizeDf(mdnrnn) as tdf: 
        pi, mu, sigma, hidden_state = mdnrnn.forward(z.unsqueeze(1).repeat((1,2,1)))
        z_next = mdnrnn.sample(pi, mu, sigma)
        df_mdnrnn = tdf.make_df()
    
    display(df_mdnrnn)
    

    with TorchSummarizeDf(finv) as tdf:
        finv(z.repeat((1,2,1)), z_next)   
        df_finv = tdf.make_df()
    display(df_finv)

    with TorchSummarizeDf(world_model) as tdf:
        world_model(gpu_img, action)
        df_world_model = tdf.make_df()
    display(df_world_model[df_world_model.level<2])
    
    del img, action, gpu_img, x, mu, z, z_next, mu_vae, pi, sigma, logvar_vae